In [1]:
import resources as r
import ipywidgets as widgets
from IPython.display import HTML, display

display(HTML('''<style>
    .widget-label { min-width: 30ex !important; }
    .widget-inline-hbox { min-width: 50ex !important; }
</style>'''))
layout = widgets.Layout(width='50%')

fluorophores, solvents, methods = r.fluorophores_solvents_methods()
basisOptions = r.Basis
methodOptions = r.Methods

job = r.Job(r.Software.orca, r.Fluorophores.az, r.Solvents.gas, r.Methods.wb97xd, r.Basis.augccpvdz, r.PCM.none, r.PCM.Eq.none, r.Fluorophores.az.root, r.Jobs.opt)

def submitCASSCF(memory:int, cores:int, time:int, partner:bool, norbs:int, nelec:int, nstates:int, useFActive:bool, basis:r.Basis, stateList:list[r.States],
               fluorophoreList:list[r.Fluorophores], submit:bool, resubmit:bool) -> None:
    with r.statusLoad('progress_gas') as df:
        with r.monarchHandler() as mon:
            for fluorophore in fluorophoreList:
                for state in stateList:
                    failed = False
                    opt_status = df.at[(fluorophore, r.Methods.wb97xd), state]
                    status = df.at[(fluorophore, r.Methods.casscf), state]
                    refJob = r.Job(r.Software.orca, fluorophore, r.Solvents.gas, r.Methods.wb97xd, r.Basis.augccpvdz, r.PCM.none, r.PCM.Eq.none, state, r.Jobs.opt)

                    if status in [None, r.Status.CASSCF.failed] or resubmit:
                        if opt_status != r.Status.SMD.finished:
                            print(f'{fluorophore.name} state {state} opt not finished')
                            failed = True
                        if useFActive:
                            if fluorophore.active == (0,0):
                                print('Fluorophore active space not defined')
                                failed = True
                            else:
                                norbs, nelec = fluorophore.active
                            
                        if not failed:
                            job = r.Job(r.Software.pyscf, fluorophore, r.Solvents.gas, r.Methods.casscf, basis , r.PCM.none, r.PCM.Eq.none, state,
                                        r.Jobs.casscfOpt, catxyzpath=refJob.xyzfile, submit=submit, partner=partner, procs=cores, mem=memory, 
                                        casscf=(nelec, norbs), time=time, nroots=nstates)
                            mon.buildJob(job)
                            print(job.name)

                            if submit:
                                df.at[(fluorophore, r.Methods.casscf), state] = r.Status.CASSCF.queued
                    elif status in [r.Status.CASSCF.finished, r.Status.CASSCF.queued, r.Status.CASSCF.running]:
                        print(f'{fluorophore.name} Orca job already submitted, use "resubmit" to overwrite the job file')
    print('Done!')

widgets.interactive(submitCASSCF, {'manual' : True, 'manual_name' : 'Submit CASSCF Job'},
                     fluorophoreList=widgets.SelectMultiple(options=fluorophores, rows=len(fluorophores), description='Fluorophore'),
                     basis=widgets.Dropdown(options=r.Basis, value=r.Basis.augccpvdz, rows=15, description='Basis'),
                     partner = widgets.Checkbox(value=False, description='Partner'),
                     stateList = widgets.SelectMultiple(options=r.States, value=[r.States.s0], description='States'),
                     submit = widgets.Checkbox(value=False, description='Submit'),
                     resubmit = widgets.Checkbox(value=False, description='ReSubmit'),
                     memory = widgets.IntText(value=64, description='Memory'),
                     cores = widgets.IntText(value=16, description='Cores'),
                     time = widgets.IntText(value=24, description='Hours'),
                     norbs = widgets.IntText(value=10, description='CASSCF Orbitals'),
                     nelec = widgets.IntText(value=10, description='CASSCF Electrons'),
                     nstates = widgets.IntText(value=10, description='Number of states'),
                     useFActive = widgets.Checkbox(value=False, description='Use Fluorophore Active Space', layout=layout),
                     )

interactive(children=(IntText(value=64, description='Memory'), IntText(value=16, description='Cores'), IntText…